In [1]:
import numpy as np
import random as rn
import findspark
import sys
import time
import numba
from numba import cuda, float32, float64
findspark.init("/opt/spark")

In [2]:
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [3]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [4]:
V_max= 10.0
W= 1.0
c_1 = 0.8
c_2 = 0.2
objetivo =[50,50,50]
ind=0
filas_x=[0,0,0]

In [5]:
class FitnessEval:
    def __init__(self):
        mejor_pos_global_=[]
        #mejor_pos_global_arr= np.array([0 for j in range(n)])
        #self.mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
        self.mejor_pos_global=np.array([0 for j in range(n)])
        self.accum=sc.accumulator(0)
        
    def fitnessEval(self, part, N):
        filas = part[0:N]
        fit = MSE(filas, objetivo)
        if fit < self.best_local_fitness[ind]:
            self.best_local_fitness[ind] = fit
            for k in range(N):
                part[2*N+k] = filas[k]
            if fit < self.best_global_fitness:
                self.best_global_fitness = fit
                self.mejor_pos_global = filas
                self.accum = fit
        if ind < 10:
            ind += 1
        return part


In [6]:
class GlobalVars:
    def __init__(self):
        mejor_pos_global_=[]
        mejor_pos_global_arr= np.array([0 for j in range(n)])
        self.mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
        self.accum=sc.accumulator(0)

In [7]:
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 100
posiciones_=[]
mejor_posiciones_locales_=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales_])
#mejor_pos_global_=[]
#mejor_pos_global_arr= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness_arr=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
best_global_fitness = max
#accum=sc.accumulator(0)
#accum=sc.accumulator(0.0)
#mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
best_local_fitness= sc.broadcast(best_local_fitness_arr)

In [8]:
def InitParticles(part,N,M,bgf,blf):
    #global best_global_fitness
    parts_ =[]
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        velocidad=[rn.uniform(-100,100) for j in range(N)]
        part_ = posicion + velocidad + posicion
        parts_.append(part_)
        
        

        fit = MSE(posicion,objetivo)
        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("bgf-> ",bgf)
        if fit < bgf:
            bgf = fit
            fit_eval.accum= fit
            fit_eval.mejor_pos_global = posicion
    parts = np.array(parts_)
    return blf,bgf,fit_eval.mejor_pos_global,parts 

In [9]:
def fitnessEval(part,N):
    global ind
    global mejor_pos_global
    #global best_global_fitness
    #global accum
    global best_local_fitness
    filas=part[0:N]
    fit = MSE(filas,objetivo)
    if fit < best_local_fitness[ind]:
        best_local_fitness[ind] = fit
        #mpl[ind] = part[0:1,ind]
        for k in range(N):
            part[2*N+k]=filas[k]
        if fit < best_global_fitness:
            #accum += (fit-best_global_fitness)
            best_global_fitness = fit
            fit_eval.mejor_pos_global = filas
            fit_eval.accum = fit
    if ind<10:
        ind+=1
    #return part,mpg,blf,bgf
    return part 

In [10]:
def posEval(part,mpg,N):
#def posEval_x(part,N):
    global ind
    velocidades=part[N:2*N]
    mpl=part[2*N:]
    r_1 = rn.random()
    r_2 = rn.random()
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[k]) + c_2*r_2*(mpg[k] - part[k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[k] = part[k] + velocidades[k]
        part[N+k] = velocidades[k]
    #return part,velocidades
    return part

In [11]:
fit_eval = FitnessEval()
best_local_fitness,best_global_fitness,fit_eval.mejor_pos_global,particulas=InitParticles(particulas,n,m,best_global_fitness,best_local_fitness)

In [12]:
rdd_master= sc.parallelize(particulas)
start = time.time()

print("rdd_master antes:", rdd_master.collect())
print("antes:",fit_eval.mejor_pos_global)
for i in range(100):
    rdd_2 = rdd_master.map(lambda x:fit_eval.fitnessEval(x,n))
    
    resultado = rdd_2.map(lambda x:posEval(x,fit_eval.mejor_pos_global,n))
    resultado = resultado.collect()
    
    for j in range(m):
        particulas[j] = resultado[j][:]
        
    rdd_master= sc.parallelize(particulas)
    
print("despues:",mejor_pos_global)
print("rdd_master despues:", rdd_master.collect())

end = time.time()
tiempo = end - start
resultado = rdd_master.collect()
print("Tiempo de ejecucion(s): ", tiempo)
print("mejor_pos_global-> ",fit_eval.mejor_pos_global)
print("mejor fitness global-> ", best_global_fitness, MSE(fit_eval.mejor_pos_global,objetivo))
print("accum:", accum.value)

rdd_master antes: [array([-35.26095841,  64.23757565, -78.32153151,  96.90964429,
       -21.62267925, -27.72085145, -35.26095841,  64.23757565,
       -78.32153151]), array([-15.34727357, -58.04714896,  97.81797104, -20.97090161,
       -88.26478399,  61.86201875, -15.34727357, -58.04714896,
        97.81797104]), array([ 65.69019705,  94.79496248,  59.5168831 , -67.24918347,
        89.79026686,  24.32955424,  65.69019705,  94.79496248,
        59.5168831 ]), array([-32.62497097,  82.23630101,  33.7178065 , -81.92843161,
        48.40187998,  27.25014737, -32.62497097,  82.23630101,
        33.7178065 ]), array([-72.6822255 , -86.31985346,   0.82667812,  35.48448986,
       -82.82460244, -74.48225711, -72.6822255 , -86.31985346,
         0.82667812]), array([ 28.40129766,  43.05843257, -66.98859888,  16.21347886,
        25.51828273, -38.19128192,  28.40129766,  43.05843257,
       -66.98859888]), array([ 39.81221437,  -3.33760285, -69.68477256, -23.94407716,
       -98.87494452, -81

Traceback (most recent call last):
  File "/opt/spark/python/pyspark/serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 102, in dumps
    cp.dump(obj)
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
  File "/usr/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/usr/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/lib/python3.6/pickle.py", line 751, in save_tuple
    save(element)
  File "/usr/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 784, in save_function
    *self._dynamic_function_reduce(obj), obj=obj
  File "/opt/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 726, in _save_r

PicklingError: Could not serialize object: RuntimeError: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.